# Fit the Random forest model

In [3]:
import pandas as pd
import numpy as np
import os
import joblib
from glob import glob

from scipy.stats import randint, uniform, loguniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

# Paths
input_folder = "data/processed/stock_data"
summary_folder = "summary"
model_folder = os.path.join(summary_folder, "random_forest_randomized")
os.makedirs(model_folder, exist_ok=True)

summary_results = []

# ---- Distributions for randomized search (wider but efficient) ----
param_distributions = {
    "n_estimators": randint(128, 257),                # small forest for stage 1
    "max_depth": [None] + list(range(5, 16)),         # shallower trees are faster
    "min_samples_leaf": randint(1, 16),
    "max_features": ["sqrt", "log2", None, uniform(0.3, 0.5)],  # categorical or fraction
    "bootstrap": [True],
    "max_samples": uniform(0.6, 0.35),                # 0.6..0.95 subsampling speeds up
    "class_weight": [None, "balanced", "balanced_subsample"]
}

# Time-series aware CV
tscv = TimeSeriesSplit(n_splits=5)

# Randomized search budget
N_ITER = 30       
RANDOM_STATE = 42

for file_path in glob(os.path.join(input_folder, "*.csv")):
    stock_name = os.path.basename(file_path).replace("_features.csv", "")
    print(f"\n=== Processing {stock_name} ===")

    # Load data
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    # Target: next-day direction
    df['Direction'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.dropna(inplace=True)

    # Features / target
    X = df.drop(columns=[
        "Date", "Close", "Target", "Direction",
        "High", "Low", "Open", "High_lag1", "Low_lag1", "Open_lag1"
    ], errors="ignore")
    y = df['Direction'].astype(int)

    # Time-ordered split
    train_size = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    # Scale (not required for RF, but kept for consistency)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Base RF
    base_rf = RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_jobs=-1,
        oob_score=True,     # useful with bootstrap=True
        # NOTE: oob_score is only computed after fit; not used by scorer but good diagnostic
    )
    from sklearn.metrics import roc_auc_score, make_scorer

    def roc_auc_safe(y_true, y_score):
        if len(np.unique(y_true)) < 2:
            return np.nan
        return roc_auc_score(y_true, y_score)


    auc_scorer = make_scorer(roc_auc_safe, needs_proba=True)

    # Randomized Search with TimeSeriesSplit; score by AUC
    rnd = RandomizedSearchCV(
        estimator=base_rf,
        param_distributions=param_distributions,
        n_iter=N_ITER,
        scoring=auc_scorer,
        cv=tscv,
        n_jobs=-1,
        verbose=1,
        random_state=RANDOM_STATE,
        refit=True,  # refit on full training set using best params
        error_score=np.nan              # keep runs going

    )

    rnd.fit(X_train_scaled, y_train)

    best_rf = rnd.best_estimator_
    print("Best params:", rnd.best_params_)
    print(f"Best CV AUC: {rnd.best_score_:.4f}")

    # Test-set evaluation
    y_pred = best_rf.predict(X_test_scaled)
    y_proba = best_rf.predict_proba(X_test_scaled)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    print(f"Random Forest (RandomizedSearch) Accuracy: {acc:.4f}, AUC: {auc:.4f}")
    print(classification_report(y_test, y_pred))

    # Save model, scaler, and best params
    model_path = os.path.join(model_folder, f"{stock_name}_rf_model.pkl")
    scaler_path = os.path.join(model_folder, f"{stock_name}_scaler.pkl")
    params_path = os.path.join(model_folder, f"{stock_name}_best_params.pkl")
    joblib.dump(best_rf, model_path)
    joblib.dump(scaler, scaler_path)
    joblib.dump(rnd.best_params_, params_path)
    print(f"Saved model to {model_path}")
    print(f"Saved scaler to {scaler_path}")
    print(f"Saved best params to {params_path}")

    # Append metrics to summary
    summary_results.append({
        "Stock": stock_name,
        "Accuracy": acc,
        "AUC": auc,
        "CV_AUC": rnd.best_score_,
        "OOB_Score": getattr(best_rf, "oob_score_", np.nan),
        "Best_n_estimators": rnd.best_params_.get("n_estimators"),
        "Best_max_depth": rnd.best_params_.get("max_depth"),
        "Best_min_samples_leaf": rnd.best_params_.get("min_samples_leaf"),
        "Best_max_features": str(rnd.best_params_.get("max_features")),
        "Best_class_weight": rnd.best_params_.get("class_weight")
    })

# Save summary
summary_df = pd.DataFrame(summary_results)
summary_file = os.path.join(summary_folder, "stock_data_model_summary_random_forest_randomized.csv")
summary_df.to_csv(summary_file, index=False)
print(f"\nSummary saved to {summary_file}")



=== Processing AAPL_daily ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 1356, in wrapper
    estimator._validate_params()
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 469, in _validate_params
    val

Best params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 7, 'max_features': 'sqrt', 'max_samples': 0.6642021764531573, 'min_samples_leaf': 8, 'n_estimators': 148}
Best CV AUC: nan
Random Forest (RandomizedSearch) Accuracy: 0.4655, AUC: 0.5108
              precision    recall  f1-score   support

           0       0.46      0.98      0.63      1040
           1       0.56      0.02      0.04      1205

    accuracy                           0.47      2245
   macro avg       0.51      0.50      0.33      2245
weighted avg       0.52      0.47      0.31      2245

Saved model to summary\random_forest_randomized\AAPL_daily_rf_model.pkl
Saved scaler to summary\random_forest_randomized\AAPL_daily_scaler.pkl
Saved best params to summary\random_forest_randomized\AAPL_daily_best_params.pkl

=== Processing GE_daily ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 1356, in wrapper
    estimator._validate_params()
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 469, in _validate_params
    val

Best params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 7, 'max_features': 'sqrt', 'max_samples': 0.6642021764531573, 'min_samples_leaf': 8, 'n_estimators': 148}
Best CV AUC: nan
Random Forest (RandomizedSearch) Accuracy: 0.4865, AUC: 0.4795
              precision    recall  f1-score   support

           0       0.48      0.38      0.42      1582
           1       0.49      0.59      0.54      1614

    accuracy                           0.49      3196
   macro avg       0.48      0.49      0.48      3196
weighted avg       0.48      0.49      0.48      3196

Saved model to summary\random_forest_randomized\GE_daily_rf_model.pkl
Saved scaler to summary\random_forest_randomized\GE_daily_scaler.pkl
Saved best params to summary\random_forest_randomized\GE_daily_best_params.pkl

=== Processing IBM_daily ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


...

Best params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 7, 'max_features': 'sqrt', 'max_samples': 0.6642021764531573, 'min_samples_leaf': 8, 'n_estimators': 148}
Best CV AUC: nan
Random Forest (RandomizedSearch) Accuracy: 0.5114, AUC: 0.4986
              precision    recall  f1-score   support

           0       0.48      0.32      0.38      1524
           1       0.53      0.69      0.60      1671

    accuracy                           0.51      3195
   macro avg       0.50      0.50      0.49      3195
weighted avg       0.50      0.51      0.49      3195

Saved model to summary\random_forest_randomized\IBM_daily_rf_model.pkl
Saved scaler to summary\random_forest_randomized\IBM_daily_scaler.pkl
Saved best params to summary\random_forest_randomized\IBM_daily_best_params.pkl

=== Processing JNJ_daily ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 1356, in wrapper
    estimator._validate_params()
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 469, in _validate_params
    val

Best params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 7, 'max_features': 'sqrt', 'max_samples': 0.6642021764531573, 'min_samples_leaf': 8, 'n_estimators': 148}
Best CV AUC: nan
Random Forest (RandomizedSearch) Accuracy: 0.4872, AUC: 0.4975
              precision    recall  f1-score   support

           0       0.47      0.74      0.58      1518
           1       0.52      0.25      0.34      1676

    accuracy                           0.49      3194
   macro avg       0.50      0.50      0.46      3194
weighted avg       0.50      0.49      0.46      3194

Saved model to summary\random_forest_randomized\JNJ_daily_rf_model.pkl
Saved scaler to summary\random_forest_randomized\JNJ_daily_scaler.pkl
Saved best params to summary\random_forest_randomized\JNJ_daily_best_params.pkl

=== Processing MSFT_daily ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 1356, in wrapper
    estimator._validate_params()
  File "c:\Users\firaas\Anaconda3\envs\stock-predict\lib\site-packages\sklearn\base.py", line 469, in _validate_params
    val

Best params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 7, 'max_features': 'sqrt', 'max_samples': 0.6642021764531573, 'min_samples_leaf': 8, 'n_estimators': 148}
Best CV AUC: nan
Random Forest (RandomizedSearch) Accuracy: 0.4919, AUC: 0.5274
              precision    recall  f1-score   support

           0       0.47      0.82      0.60       903
           1       0.59      0.22      0.32      1077

    accuracy                           0.49      1980
   macro avg       0.53      0.52      0.46      1980
weighted avg       0.53      0.49      0.44      1980

Saved model to summary\random_forest_randomized\MSFT_daily_rf_model.pkl
Saved scaler to summary\random_forest_randomized\MSFT_daily_scaler.pkl
Saved best params to summary\random_forest_randomized\MSFT_daily_best_params.pkl

Summary saved to summary\stock_data_model_summary_random_forest_randomized.csv


# Test the Random forest model

In [4]:
import pandas as pd
import numpy as np
import os
import joblib
from glob import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

# ---------------------- Config ----------------------
INPUT_FOLDER = "data/processed/stock_data"
SUMMARY_FOLDER = "summary"
MODEL_FOLDER = os.path.join(SUMMARY_FOLDER, "random_forest_randomized")
RESULTS_FOLDER = os.path.join(SUMMARY_FOLDER, "donchian_eval_rf")
os.makedirs(MODEL_FOLDER, exist_ok=True)
os.makedirs(RESULTS_FOLDER, exist_ok=True)

TRADING_DAYS = 252
RISK_FREE_ANNUAL = 0.0  # set e.g. 0.03 for 3% annual
RF_CONFIDENCE_THRESHOLD = 0.0  # 0..1 (based on |p-0.5|*2)
DONCHIAN_PERIOD = 20
N_PERMUTATIONS = 500  # increase to 1000+ for tighter p-values

# ---------------------- Helpers ----------------------
def annualized_sharpe(returns, rf_annual=0.0, periods_per_year=252):
    if len(returns) == 0 or returns.std() == 0:
        return 0.0
    rf_daily = rf_annual / periods_per_year
    return (returns.mean() - rf_daily) / returns.std() * np.sqrt(periods_per_year)

def sortino_ratio(returns, target=0.0, periods_per_year=252):
    if len(returns) == 0:
        return 0.0
    # downside deviation relative to target (per-period)
    downside = returns[returns < target]
    if len(downside) == 0:
        return np.inf  # no downside volatility
    downside_std = downside.std()
    if downside_std == 0:
        return np.inf
    # annualize like Sharpe (mean excess over target / downside std)
    return (returns.mean() - target) / downside_std * np.sqrt(periods_per_year)

def calculate_profit_factor(returns):
    gains = returns[returns > 0].sum()
    losses = returns[returns < 0].sum()
    losses = abs(losses)
    if losses == 0:
        return np.inf if gains > 0 else 0.0
    return gains / losses

def donchian_breakout_strategy(df, predictions, confidence_scores,
                               donchian_period=20, confidence_threshold=0.0):
    """
    Long when breakout up AND model predicts up with confidence.
    Short when breakdown AND model predicts down with confidence.
    Position is held until the next signal (FFILL).
    """
    d = df.copy()
    d["Pred"] = np.array(predictions)[:len(d)]
    d["Conf"] = np.array(confidence_scores)[:len(d)]

    d["Donchian_High"] = d["High"].rolling(donchian_period).max()
    d["Donchian_Low"] = d["Low"].rolling(donchian_period).min()

    d["Long_Signal"] = (d["Close"] > d["Donchian_High"].shift(1)) & (d["Pred"] == 1) & (d["Conf"] > confidence_threshold)
    d["Short_Signal"] = (d["Close"] < d["Donchian_Low"].shift(1)) & (d["Pred"] == 0) & (d["Conf"] > confidence_threshold)

    d["Position"] = 0
    d.loc[d["Long_Signal"], "Position"] = 1
    d.loc[d["Short_Signal"], "Position"] = -1
    d["Position"] = d["Position"].replace(0, np.nan).ffill().fillna(0)

    d["Market_Return"] = d["Close"].pct_change()
    d["Strategy_Return"] = d["Position"].shift(1) * d["Market_Return"]
    d = d.dropna().copy()
    return d

def monte_carlo_pf_pvalue(df, predictions, confidence_scores,
                          donchian_period=20, confidence_threshold=0.0,
                          n_permutations=500):
    """Permutation test on Profit Factor (one-tailed: random PF >= actual PF)."""
    actual_df = donchian_breakout_strategy(df, predictions, confidence_scores,
                                           donchian_period, confidence_threshold)
    actual = actual_df["Strategy_Return"]
    if len(actual) == 0:
        return 0.0, np.array([0.0]), 1.0
    actual_pf = calculate_profit_factor(actual)

    rnd_pfs = []
    for _ in range(n_permutations):
        # shuffle predictions but keep confidences and prices intact
        shuffled = np.random.permutation(predictions)
        rnd_df = donchian_breakout_strategy(df, shuffled, confidence_scores,
                                            donchian_period, confidence_threshold)
        r = rnd_df["Strategy_Return"]
        pf = calculate_profit_factor(r) if len(r) else 0.0
        rnd_pfs.append(pf)

    rnd_pfs = np.array(rnd_pfs)
    p_val = (rnd_pfs >= actual_pf).mean()
    return actual_pf, rnd_pfs, p_val

# ---------------------- Main Loop ----------------------
summary_rows = []

for file_path in glob(os.path.join(INPUT_FOLDER, "*.csv")):
    stock_name = os.path.basename(file_path).replace("_features.csv", "")
    print(f"\n=== Processing {stock_name} ===")

    # Load full data
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    # Binary target (next-day direction)
    df["Direction"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
    df.dropna(inplace=True)

    # Train/test split (time‑ordered)
    features_to_drop = [
        "Date", "Close", "Target", "Direction",
        "High", "Low", "Open", "High_lag1", "Low_lag1", "Open_lag1"
    ]
    X = df.drop(columns=features_to_drop, errors="ignore")
    y = df["Direction"].astype(int)

    split = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:split], X.iloc[split:]
    y_train, y_test = y.iloc[:split], y.iloc[split:]

    # Keep price columns for strategy evaluation on test set
    test_df = df.iloc[split:].copy()

    # Scale (optional for RF; kept for consistency)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train RF
    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        n_jobs=-1,
        class_weight=None
    )
    rf.fit(X_train_scaled, y_train)

    # Predict
    y_pred = rf.predict(X_test_scaled)
    y_proba = rf.predict_proba(X_test_scaled)[:, 1]
    acc = accuracy_score(y_test, y_pred)
    try:
        auc = roc_auc_score(y_test, y_proba)
    except ValueError:
        auc = np.nan
    auc_str = f"{auc:.4f}" if not np.isnan(auc) else "nan"
    print(f"RF Acc={acc:.4f}, AUC={auc_str}")
    # print(classification_report(y_test, y_pred))  # optional

    # Save model + scaler per stock (optional)
    model_path = os.path.join(MODEL_FOLDER, f"{stock_name}_rf_model.pkl")
    scaler_path = os.path.join(MODEL_FOLDER, f"{stock_name}_scaler.pkl")
    joblib.dump(rf, model_path)
    joblib.dump(scaler, scaler_path)

    # Confidence from probabilities (0..1): distance from 0.5, scaled to 0..1
    confidence_scores = np.abs(y_proba - 0.5) * 2.0

    # Run strategy on the test window aligned with predictions
    strat_df = donchian_breakout_strategy(
        df=test_df,
        predictions=y_pred,
        confidence_scores=confidence_scores,
        donchian_period=DONCHIAN_PERIOD,
        confidence_threshold=RF_CONFIDENCE_THRESHOLD
    )

    # Metrics
    strat_ret = strat_df["Strategy_Return"]
    mkt_ret = strat_df["Market_Return"]

    total_strat_return = (1 + strat_ret).prod() - 1 if len(strat_ret) else 0.0
    total_mkt_return = (1 + mkt_ret).prod() - 1 if len(mkt_ret) else 0.0

    sharpe = annualized_sharpe(strat_ret, rf_annual=RISK_FREE_ANNUAL, periods_per_year=TRADING_DAYS)
    mkt_sharpe = annualized_sharpe(mkt_ret, rf_annual=RISK_FREE_ANNUAL, periods_per_year=TRADING_DAYS)

    sortino = sortino_ratio(strat_ret, target=0.0, periods_per_year=TRADING_DAYS)
    mkt_sortino = sortino_ratio(mkt_ret, target=0.0, periods_per_year=TRADING_DAYS)

    pf = calculate_profit_factor(strat_ret)
    mkt_pf = calculate_profit_factor(mkt_ret)

    # Permutation test on PF
    actual_pf, rnd_pfs, p_val = monte_carlo_pf_pvalue(
        test_df,
        y_pred,
        confidence_scores,
        donchian_period=DONCHIAN_PERIOD,
        confidence_threshold=RF_CONFIDENCE_THRESHOLD,
        n_permutations=N_PERMUTATIONS
    )

    # Basic trade stats
    num_trades = (strat_df["Position"].diff().abs().fillna(0).sum()) / 2
    win_rate = (strat_ret > 0).mean() if len(strat_ret) else 0.0

    print(
        f"  → Return={total_strat_return:.3f} | PF={pf:.3f} | Sharpe={sharpe:.2f} | "
        f"Sortino={sortino:.2f} | p={p_val:.3f} | Trades={int(num_trades)} | Win%={win_rate:.2%}"
    )

    summary_rows.append({
        "Stock": stock_name,
        "Test_Samples": len(test_df),
        "Accuracy": acc,
        "AUC": auc,
        "Total_Strategy_Return": total_strat_return,
        "Total_Market_Return": total_mkt_return,
        "Excess_Return": total_strat_return - total_mkt_return,
        "Profit_Factor": pf,
        "Market_Profit_Factor": mkt_pf,
        "Sharpe": sharpe,
        "Market_Sharpe": mkt_sharpe,
        "Sortino": sortino,
        "Market_Sortino": mkt_sortino,
        "Win_Rate": win_rate,
        "Num_Trades": int(num_trades),
        "PF_p_value": p_val,
        "Test_Start": str(test_df["Date"].iloc[0]) if "Date" in test_df.columns else "",
        "Test_End": str(test_df["Date"].iloc[-1]) if "Date" in test_df.columns else ""
    })

# ---------------------- Save + Leaderboards ----------------------
summary_df = pd.DataFrame(summary_rows)
summary_csv = os.path.join(RESULTS_FOLDER, "donchian_rf_eval_summary.csv")
summary_df.to_csv(summary_csv, index=False)
print(f"\nSaved evaluation summary to: {summary_csv}")

if not summary_df.empty:
    print("\nTop 5 by Profit Factor:")
    for _, r in summary_df.nlargest(5, "Profit_Factor").iterrows():
        print(f"  - {r['Stock']}: PF={r['Profit_Factor']:.3f}, p={r['PF_p_value']:.3f}, Ret={r['Total_Strategy_Return']:.3f}")

    print("\nTop 5 by Sharpe:")
    for _, r in summary_df.nlargest(5, "Sharpe").iterrows():
        print(f"  - {r['Stock']}: Sharpe={r['Sharpe']:.2f}, Sortino={r['Sortino']:.2f}, Ret={r['Total_Strategy_Return']:.3f}")

    print("\nStatistically significant PF (p < 0.05):")
    sig = summary_df[summary_df["PF_p_value"] < 0.05]
    if len(sig):
        for _, r in sig.sort_values("PF_p_value").iterrows():
            print(f"  - {r['Stock']}: PF={r['Profit_Factor']:.3f}, p={r['PF_p_value']:.4f}, Ret={r['Total_Strategy_Return']:.3f}")
    else:
        print("  (none)")



=== Processing AAPL_daily ===
RF Acc=0.4833, AUC=0.5103
  → Return=-0.618 | PF=0.960 | Sharpe=-0.22 | Sortino=-0.30 | p=0.600 | Trades=16 | Win%=46.50%

=== Processing GE_daily ===
RF Acc=0.4972, AUC=0.4876
  → Return=-0.160 | PF=1.023 | Sharpe=0.12 | Sortino=0.16 | p=0.868 | Trades=29 | Win%=50.27%

=== Processing IBM_daily ===
RF Acc=0.5002, AUC=0.5033
  → Return=-0.810 | PF=0.919 | Sharpe=-0.44 | Sortino=-0.59 | p=0.882 | Trades=47 | Win%=48.14%

=== Processing JNJ_daily ===
RF Acc=0.4834, AUC=0.4991
  → Return=-0.816 | PF=0.879 | Sharpe=-0.68 | Sortino=-0.94 | p=0.900 | Trades=11 | Win%=44.91%

=== Processing MSFT_daily ===
RF Acc=0.4833, AUC=0.5171
  → Return=-0.855 | PF=0.842 | Sharpe=-0.83 | Sortino=-1.00 | p=0.958 | Trades=0 | Win%=35.49%

Saved evaluation summary to: summary\donchian_eval_rf\donchian_rf_eval_summary.csv

Top 5 by Profit Factor:
  - GE_daily: PF=1.023, p=0.868, Ret=-0.160
  - AAPL_daily: PF=0.960, p=0.600, Ret=-0.618
  - IBM_daily: PF=0.919, p=0.882, Ret=-0.81